In [2]:
import pandas as pd
from typing import Dict, List, Union, Optional

def check_duplicate_events(df: pd.DataFrame) -> Dict:
    """Проверка дубликатов событий с одинаковыми ts и event"""
    results = {'duplicates': []}
    duplicate_cols = ['ts', 'event']

    if 'counter_id' in df.columns:
        duplicate_cols.append('counter_id')

    duplicates = df[df.duplicated(subset=duplicate_cols, keep=False)]
    if not duplicates.empty:
        results['duplicates'] = duplicates.sort_values(duplicate_cols).to_dict('records')
    return results

print(check_duplicate_events(df))
def check_session_start(df: pd.DataFrame) -> Dict:
    """Проверка, что сессии начинаются с 1"""
    results = {'start_errors': []}

    if 'randPAS_session_id' not in df.columns:
        raise KeyError("Отсутствует столбец randPAS_session_id")

    first_events = df.groupby('randPAS_session_id').first()

    start_errors = first_events[
        (first_events['page_view_order_number'] != 1) |
        (first_events['event_order_number'] != 1)
    ]

    for session_id, row in start_errors.iterrows():
        results['start_errors'].append({
            'session_id': session_id,
            'first_page_view': int(row['page_view_order_number']),
            'first_event': int(row['event_order_number'])
        })

    return results

print(ckeck_session_start(df))

def check_order_relation(df: pd.DataFrame) -> Dict:
    """Проверка соотношения page_view и event order numbers"""
    results = {'relation_errors': []}

    relation_errors = df[df['event_order_number'] < df['page_view_order_number']]

    if not relation_errors.empty:
        for session_id, group in relation_errors.groupby('randPAS_session_id'):
            results['relation_errors'].append({
                'session_id': session_id,
                'count': len(group),
                'examples': group[['ts', 'page_view_order_number', 'event_order_number']].head(3).to_dict('records')
            })

    return results



def check_numbering_sequence(df: pd.DataFrame) -> Dict:
    """Проверка пропусков в нумерации событий"""
    results = {'missing_numbers': []}

    grouped = df.groupby('randPAS_session_id')

    for session_id, group in grouped:
        session_data = group.sort_values('ts')

        page_diff = session_data['page_view_order_number'].diff().dropna()
        if any(page_diff != 1):
            results['missing_numbers'].append({
                'session_id': session_id,
                'type': 'page_view',
                'positions': session_data[page_diff != 1][['ts', 'page_view_order_number']].to_dict('records')
            })


        event_diff = session_data['event_order_number'].diff().dropna()
        if any(event_diff != 1):
            results['missing_numbers'].append({
                'session_id': session_id,
                'type': 'event',
                'positions': session_data[event_diff != 1][['ts', 'event_order_number']].to_dict('records')
            })

    return results


!gdown --id 1GvWVG9iS3sQkYnaERcd_G2hCbpa2xLyC
import pandas as pd

df = pd.read_parquet('data_2024-10-09_part2.parquet')


import pandas as pd
from datetime import timedelta

def detect_location_changes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Анализирует смену местоположения (geo_city_id и ip) для каждого пользователя.
    Возвращает DataFrame с информацией о сменах местоположения и временных интервалах.
    """
    df_sorted = df.sort_values(['randPAS_user_passport_id', 'ts'])
    grouped = df_sorted.groupby('randPAS_user_passport_id')

    results = []

    for user_id, group in grouped:
        user_data = group[['ts', 'ip', 'geo_city_id']].drop_duplicates()

        changes = user_data[
            (user_data['geo_city_id'].shift() != user_data['geo_city_id']) |
            (user_data['ip'].shift() != user_data['ip'])
        ].copy()
        if len(changes) > 1:
            changes['time_diff'] = changes['ts'].diff().dt.total_seconds()
            city_changes = list(zip(
                changes['geo_city_id'].astype(str),
                changes['time_diff'].astype(str)
            ))

            results.append({
                'user_id': user_id,
                'city_changes': " → ".join([f"{city} ({time}s)" for city, time in city_changes]),
                'change_count': len(changes) - 1,
                'first_change': changes['ts'].iloc[1],
                'last_change': changes['ts'].iloc[-1],
                'unique_cities': changes['geo_city_id'].nunique(),
                'unique_ips': changes['ip'].nunique()
            })

    return pd.DataFrame(results)

if __name__ == "__main__":

    test_data = {
        'ts': pd.to_datetime(['2023-01-01 10:00', '2023-01-01 11:00', '2023-01-01 12:00',
                             '2023-01-02 09:00', '2023-01-02 10:00']),
        'randPAS_user_passport_id': ['user1', 'user1', 'user1', 'user2', 'user2'],
        'ip': ['192.168.1.1', '192.168.1.2', '192.168.1.2', '10.0.0.1', '10.0.0.2'],
        'geo_city_id': [1, 2, 5, 3, 4]
    }
    df = pd.DataFrame(test_data)

    result_df = detect_location_changes(df)

    print("Анализ смены местоположения пользователей:")
    print(result_df[['user_id', 'city_changes', 'change_count']])

    if not result_df.empty:
        print("\nСтатистика по изменениям:")
        print(f"Всего пользователей с изменениями: {len(result_df)}")
        print(f"Среднее количество изменений на пользователя: {result_df['change_count'].mean():.2f}")
        print(f"Максимальное количество городов у одного пользователя: {result_df['unique_cities'].max()}")


import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def analyze_city_activity(df, min_events=10, z_threshold=3, rolling_window='1H'):
    """
    Анализирует активность по городам и выявляет аномальные всплески

    Параметры:
        df: DataFrame с данными
        min_events: минимальное количество событий для анализа города
        z_threshold: порог для детектирования аномалий (в стандартных отклонениях)
        rolling_window: размер окна для скользящей статистики

    Возвращает:
        DataFrame с результатами анализа
    """

    if not pd.api.types.is_datetime64_any_dtype(df['ts']):
        df['ts'] = pd.to_datetime(df['ts'])
    city_activity = df.groupby(['geo_city_id', pd.Grouper(key='ts', freq=rolling_window)])\
                     .size()\
                     .reset_index(name='event_count')
    #print(city_activity)
    city_stats = city_activity.groupby('geo_city_id')['event_count'].agg(['count', 'mean', 'std'])
    valid_cities = city_stats[city_stats['count'] > 5].index
    city_activity = city_activity[city_activity['geo_city_id'].isin(valid_cities)]

    city_activity['z_score'] = city_activity.groupby('geo_city_id')['event_count']\
        .transform(lambda x: (x - x.mean()) / x.std())

    city_activity['is_anomaly'] = city_activity['z_score'] > z_threshold


    total_activity = df.groupby(pd.Grouper(key='ts', freq=rolling_window))\
                      .size()\
                      .reset_index(name='total_events')

    city_activity = city_activity.merge(total_activity, on='ts')
    city_activity['activity_ratio'] = city_activity['event_count'] / city_activity['total_events']

    city_activity['prev_ratio'] = city_activity.groupby('geo_city_id')['activity_ratio'].shift(1)
    city_activity['ratio_change'] = (city_activity['activity_ratio'] - city_activity['prev_ratio']) / city_activity['prev_ratio']

    anomalies = city_activity[city_activity['is_anomaly']].sort_values('z_score', ascending=False)

    return anomalies, city_activity



anomalies, city_activity = analyze_city_activity(df)

print("Топ аномалий активности:")
print(anomalies[['ts', 'geo_city_id', 'event_count', 'z_score', 'activity_ratio', 'ratio_change']].head())

import pandas as pd
from datetime import timedelta

def detect_suspicious_ips(df: pd.DataFrame,
                         max_users_per_ip: int = 10) -> pd.DataFrame:
    """
    Обнаруживает IP-адреса с аномально большим количеством пользователей

    Параметры:
        df: DataFrame с данными
        max_users_per_ip: максимальное допустимое количество пользователей с одного IP

    Возвращает:
        DataFrame с подозрительными IP и статистикой
    """
    ip_stats = (
        df.groupby('ip')
        .agg(
            unique_users=('randPAS_user_passport_id', 'nunique'),
            total_actions=('randPAS_user_passport_id', 'count'),
            first_seen=('ts', 'min'),
            last_seen=('ts', 'max')
        )
        .reset_index()
    )

    suspicious_ips = ip_stats[ip_stats['unique_users'] > max_users_per_ip]
    suspicious_ips['activity_period'] = suspicious_ips['last_seen'] - suspicious_ips['first_seen']

    return suspicious_ips.sort_values('unique_users', ascending=False)


def detect_user_activity_spikes(df: pd.DataFrame,
                              time_window_sec: int = 60,
                              max_actions: int = 30) -> pd.DataFrame:
    """
    Обнаруживает пользователей с аномально высокой активностью

    Параметры:
        df: DataFrame с данными
        time_window_sec: временное окно в секундах для анализа
        max_actions: максимальное допустимое количество действий за окно

    Возвращает:
        DataFrame с подозрительными пользователями и статистикой
    """

    df_sorted = df.sort_values(['randPAS_user_passport_id', 'ts'])
    df_sorted['time_diff'] = (
        df_sorted.groupby('randPAS_user_passport_id')['ts']
        .diff()
        .dt.total_seconds()
    )
    rapid_actions = df_sorted[df_sorted['time_diff'] < time_window_sec] \
    .groupby('randPAS_user_passport_id') \
    .agg(
        rapid_actions_count=('time_diff', 'count'),
        min_time_diff=('time_diff', 'min'),
        avg_time_diff=('time_diff', 'mean'),
        ip_list=('ip', lambda x: x.unique().tolist())
    ) \
    .reset_index()
    suspicious_users = rapid_actions[rapid_actions['rapid_actions_count'] > max_actions]
    suspicious_users['ip_count'] = suspicious_users['ip_list'].apply(len)

    return suspicious_users.sort_values('rapid_actions_count', ascending=False)



print("Анализ подозрительных IP-адресов:")
suspicious_ips = detect_suspicious_ips(df, max_users_per_ip=10)
print(suspicious_ips)

print("\nАнализ подозрительной активности пользователей:")
suspicious_users = detect_user_activity_spikes(df, time_window_sec=10, max_actions=5)
print(suspicious_users)

def detect_anomalous_time_windows(df, time_col, user_col, threshold=1.5, window_size='1H'):
    df = df.copy()

    df[time_col] = pd.to_datetime(df[time_col])
    df['time_window'] = df[time_col].dt.floor(window_size)

    df = df.sort_values(by=[user_col, time_col])
    df['time_diff'] = df.groupby(user_col)[time_col].diff().dt.total_seconds()

    df['time_category'] = pd.cut(df['time_diff'],
                                 bins=[0, 30, 300, 1800, float('inf')],
                                 labels=['short', 'medium', 'long', 'very_long'])

    time_window_stats = df.groupby('time_window')['time_category'].value_counts(normalize=True).unstack().fillna(0)

    time_window_stats['short_ratio_change'] = time_window_stats['short'].pct_change().abs().fillna(0)

    anomalous_windows = time_window_stats[time_window_stats['short_ratio_change'] > threshold]

    return anomalous_windows[['short', 'medium', 'long']]

# смотрим доли действий, разница между которыми до 30 секунд, до 5 минут и до 30 минут. если резко меняется, то выводим.
anomalies = detect_anomalous_time_windows(df, time_col='ts', user_col='randPAS_user_passport_id')

print("Аномальные временные окна с резкими изменениями долей:")
print(anomalies)
import pandas as pd

def detect_anomalous_device_shares(df, time_col, device_col, threshold=1.5, window_size='30T'):
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col])
    df['time_window'] = df[time_col].dt.floor(window_size)

    device_shares = df.groupby(['time_window', device_col]).size().unstack().fillna(0)
    device_shares = device_shares.div(device_shares.sum(axis=1), axis=0)
    device_shares_change = device_shares.pct_change().abs().fillna(0)

    anomalous_windows = device_shares_change[device_shares_change.max(axis=1) > threshold]
    return anomalous_windows

#аномалии во временных окнах по распределению типов устройств
anomalous_device_windows = detect_anomalous_device_shares(df, time_col='ts', device_col='ua_device_type')
print("Аномальные временные окна с резкими изменениями в долях устройств:")
print(anomalous_device_windows)

def detect_anomalous_page_views(df, time_col, user_col, page_col, threshold=3, window_size='30T'):
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col])
    df['time_window'] = df[time_col].dt.floor(window_size)
    page_views = df.groupby(['time_window', page_col])[user_col].nunique().unstack().fillna(0)
    page_views_change = page_views.pct_change().abs()
    page_views_change[page_views.shift(1) == 0] = np.nan
    anomalies = page_views_change[page_views_change > threshold].stack().reset_index()
    anomalies.columns = ['time_window', 'url', 'growth']

    return anomalies.dropna()

anomalous_page_windows = detect_anomalous_page_views(
    df,
    time_col='ts',
    user_col='randPAS_user_passport_id',
    page_col='url'
)

if not anomalous_page_windows.empty:
    print(" Найдены аномальные всплески посещаемости страниц:")
    for _, row in anomalous_page_windows.iterrows():
        print(f" {row['time_window']} |  {row['url']} |  Рост в {row['growth']:.2f} раз")
else:
    print("Аномалий не найдено")


from sklearn.ensemble import IsolationForest
import pandas as pd

def detect_anomalous_users(df, user_col, time_col, page_col):
    user_page_times = df.groupby([user_col, page_col])[time_col].sum().reset_index()

    user_avg_times = user_page_times.groupby(user_col)[time_col].mean().reset_index()
    user_avg_times.columns = [user_col, 'avg_time_spent']
    model = IsolationForest(contamination=0.05, random_state=42)  # contamination - доля аномальных
    user_avg_times['is_anomalous'] = model.fit_predict(user_avg_times[['avg_time_spent']])
    anomalous_users = user_avg_times[user_avg_times['is_anomalous'] == -1]

    return anomalous_users


anomalous_users = detect_anomalous_users(df, user_col='randPAS_user_passport_id', time_col='secs', page_col='url')

if not anomalous_users.empty:
    print("Найдены аномальные пользователи с нетипичным поведением:")
    print(anomalous_users)
else:
    print("Аномальных пользователей не найдено")

     import numpy as np
from scipy import stats
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

def zscore_detector(data: np.ndarray, threshold: float = 3.0) -> np.ndarray:
    """
    Обнаружение аномалий с помощью Z-Score.

    Параметры:
        data: одномерный массив данных
        threshold: пороговое значение (в стандартных отклонениях)

    Возвращает:
        Бинарный массив (1 - аномалия, 0 - норма)
    """
    z_scores = np.abs(stats.zscore(data))
    return (z_scores > threshold).astype(int)

def iqr_detector(data: np.ndarray, k: float = 1.5) -> np.ndarray:
    """
    Обнаружение аномалий с помощью межквартильного размаха (IQR).

    Параметры:
        data: одномерный массив данных
        k: множитель IQR (обычно 1.5)

    Возвращает:
        Бинарный массив (1 - аномалия, 0 - норма)
    """
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - k * iqr
    upper_bound = q3 + k * iqr
    return ((data < lower_bound) | (data > upper_bound)).astype(int)

def modified_zscore_detector(data: np.ndarray, threshold: float = 3.5) -> np.ndarray:
    """
    Обнаружение аномалий с помощью модифицированного Z-Score (более устойчив к выбросам).

    Параметры:
        data: одномерный массив данных
        threshold: пороговое значение

    Возвращает:
        Бинарный массив (1 - аномалия, 0 - норма)
    """
    median = np.median(data)
    mad = np.median(np.abs(data - median))
    modified_z = 0.6745 * (data - median) / mad
    return (np.abs(modified_z) > threshold).astype(int)

def isolation_forest_detector(data: np.ndarray,
                            contamination: float = 0.05) -> np.ndarray:
    """
    Обнаружение аномалий с помощью Isolation Forest.

    Параметры:
        data: одномерный массив данных
        contamination: предполагаемая доля аномалий

    Возвращает:
        Бинарный массив (1 - аномалия, 0 - норма)
    """
    clf = IsolationForest(contamination=contamination, random_state=42)
    data_reshaped = data.reshape(-1, 1)
    preds = clf.fit_predict(data_reshaped)
    return (preds == -1).astype(int)

def lof_detector(data: np.ndarray,
                n_neighbors: int = 20,
                contamination: float = 0.05) -> np.ndarray:
    """
    Обнаружение аномалий с помощью Local Outlier Factor (LOF).

    Параметры:
        data: одномерный массив данных
        n_neighbors: количество соседей для анализа
        contamination: предполагаемая доля аномалий

    Возвращает:
        Бинарный массив (1 - аномалия, 0 - норма)
    """
    lof = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
    data_reshaped = data.reshape(-1, 1)
    preds = lof.fit_predict(data_reshaped)
    return (preds == -1).astype(int)

def percentile_detector(data: np.ndarray,
                       lower_percentile: float = 1,
                       upper_percentile: float = 99) -> np.ndarray:
    """
    Обнаружение аномалий по перцентилям.

    Параметры:
        data: одномерный массив данных
        lower_percentile: нижний перцентиль
        upper_percentile: верхний перцентиль

    Возвращает:
        Бинарный массив (1 - аномалия, 0 - норма)
    """
    lower_bound = np.percentile(data, lower_percentile)
    upper_bound = np.percentile(data, upper_percentile)
    return ((data < lower_bound) | (data > upper_bound)).astype(int)

import numpy as np

def majority_anomaly_vote(*anomaly_arrays: np.ndarray) -> np.ndarray:
    """
    Объединяет результаты нескольких методов обнаружения аномалий.
    Аномалия отмечается, если более половины методов считают точку аномальной.

    Параметры:
        *anomaly_arrays: любое количество бинарных массивов (0 - норма, 1 - аномалия)

    Возвращает:
        Бинарный массив (1 - аномалия, 0 - норма)
    """
    if len(anomaly_arrays) == 0:
        raise ValueError("Не передано ни одного массива аномалий.")
    anomaly_matrix = np.vstack(anomaly_arrays)
    anomaly_counts = np.sum(anomaly_matrix, axis=0)
    threshold = len(anomaly_arrays) / 2
    final_anomalies = (anomaly_counts > threshold).astype(int)

    return final_anomalies



import numpy as np
import matplotlib.pyplot as plt

def plot_anomalies_comparison(data, **anomaly_dict):
    """
    Функция для построения графика сравнения методов обнаружения аномалий.

    Параметры:
        data: массив исходных данных
        anomaly_dict: именованные массивы аномалий (где 1 - аномалия, 0 - нет)
    """
    plt.figure(figsize=(12, 6))
    plt.scatter(range(len(data)), data, c='blue', label='Данные', alpha=0.5)

    # Перебираем методы и отображаем найденные аномалии
    for method_name, anomalies in anomaly_dict.items():
        anomaly_indices = np.where(anomalies == 1)[0]
        plt.scatter(anomaly_indices, data[anomaly_indices], label=method_name, alpha=0.6)

    plt.legend()
    plt.title("Сравнение методов обнаружения аномалий")
    plt.show()
if __name__ == "__main__":
    # Создаем тестовые данные
    np.random.seed(42)
    normal_data = np.random.normal(0, 1, 1000)
    anomalies = np.array([5, -5, 10, -8, 12])  # Явные аномалии
    data = np.concatenate([normal_data, anomalies])

    # Применяем детекторы
    methods = {
        "Z-Score": zscore_detector,
        "IQR": iqr_detector,
        "Modified Z-Score": modified_zscore_detector,
        "Isolation Forest": isolation_forest_detector,
        "LOF": lof_detector,
        "Percentile": percentile_detector
    }

    results = {name: detector(data) for name, detector in methods.items()}

    # Выводим количество обнаруженных аномалий
    print("Количество обнаруженных аномалий:")
    for name, anomalies in results.items():
        print(f"{name}: {np.sum(anomalies)}")

    # Визуализация с новой функцией
    plot_anomalies_comparison(data, **results)


    res = []
    for i, (name, anomalies) in enumerate(results.items(), 1):
        res.append(anomalies)

    print(len(res))
    final_anomalies = majority_anomaly_vote(res[0], res[1], res[2], res[3], res[4], res[5])

    print("Обнаружено аномалий (по большинству методов):", np.sum(final_anomalies))




NameError: name 'df' is not defined